In [1]:
import os, types, requests, base64, json
import pandas as pd
from openai import AzureOpenAI
from tqdm import tqdm
from dotenv import load_dotenv
# Carregando as variáveis de ambiente do arquivo .env
load_dotenv()

True

In [2]:
def verificar_variaveis(*nomes_variaveis):
    """
    Verifica se as variáveis de ambiente estão carregadas.

    Args:
    *nomes_variaveis: Lista de nomes das variáveis de ambiente a serem verificadas.
    """
    for nome_variavel in nomes_variaveis:
        valor_variavel = os.getenv(nome_variavel)
        if valor_variavel:
            print(f"{nome_variavel} carregada com sucesso")
        else:
            print(f"{nome_variavel} não encontrada no arquivo .env.")
            

# Acessando a variável de ambiente
api_key = os.getenv("API_KEY")
#endpoint = os.getenv("ENDPOINT_URL")
endpoint = "https://teste-genai.openai.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2024-02-15-preview" # MINI
#endpoint = "https://teste-genai.openai.azure.com/openai/deployments/GPT-4o/chat/completions?api-version=2024-02-15-preview"  # 4o
#image = os.getenv("IMAGE_PATH")
# Lista de variáveis de ambiente a verificar
variaveis_para_verificar = ["API_KEY", "ENDPOINT_URL", "IMAGE_PATH"]

# Chama a função de verificação para as variáveis
verificar_variaveis(*variaveis_para_verificar)

API_KEY carregada com sucesso
ENDPOINT_URL carregada com sucesso
IMAGE_PATH carregada com sucesso


In [3]:
# Configuration
GPT4V_KEY = api_key
GPT4V_ENDPOINT = endpoint
#IMAGE_PATH = image
#encoded_image = base64.b64encode(open(IMAGE_PATH, 'rb').read()).decode('ascii')
headers = {
    "Content-Type": "application/json",
    "api-key": GPT4V_KEY,
}


In [4]:
df_task = pd.read_excel('base_cessna V2.xlsx')

In [5]:
df_task.head(5)

,TASK
0,"Interior Placards, Exterior Placards, Decals, ..."
1,"Ventilation System - Inspect clamps, hoses, an..."
2,Primary Flight Display (PFD) Fan and Multi-Fun...
3,"Cold and Hot Air Hoses - Check condition, rout..."
4,"Heater Components, Inlets, and Outlets - Inspe..."


# Funções

In [7]:
# Payload for the request
payload_incorreto = {
  "messages": [
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": "You're the new maintenance inspector at an aviation company.\n\nAs you're new to the job, you're still making mistakes and don't know exactly all the correct procedures.\n\nI'm going to go through inspection items from an aircraft maintenance plan and I want you to generate an incorrect inspection description based on the text I've given you.\nYou can't answer like a relaxed person who hasn't bothered to look properly.\nUse the item I've given you as a basis, but consider that you didn't have access to the maintenance plan, but didn't know you were doing it wrong.\n\nAll texts must be answered impersonally."
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "Ventilation System - Inspect clamps, hoses, and valves for condition and security."
        }
      ]
    },
    {
      "role": "assistant",
      "content": [
        {
          "type": "text",
          "text": "Inspected the clamps, cables, power panels and the general condition of the ventilation system."
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "Interior Placards, Exterior Placards, Decals, Markings and Identification Plates - Examine for correct installation and legibility. Refer to Chapter 11 Placards and Markings - Inspection/Check.\n"
        }
      ]
    },
    {
      "role": "assistant",
      "content": [
        {
          "type": "text",
          "text": "Visual inspection of external and identification Placards. Correct installation criteria checked using chapter 13.\n"
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "Cold and Hot Air Hoses - Check condition, routing, and security.\n"
        }
      ]
    },
    {
      "role": "assistant",
      "content": [
        {
          "type": "text",
          "text": "Inspect the general area around the air hoses. Check that there are no visible leaks. Nothing unusual observed."
        }
      ]
    }
  ],
  "temperature": 0.9,
  "top_p": 0.95,
  "max_tokens": 800
}

In [8]:
# prompts = []

# for input_text in df_task.TASK:
    
# # Crie a string de instrução
#     test_instr = {
#         "role": "user",
#         "content": input_text
#     }
#     payload_temp = payload_incorreto.copy()
#     payload_temp['messages'] = payload_temp["messages"] + [test_instr]

#     prompts.append(payload_temp)

In [6]:

def gerar_prompt(df, coluna, payload):
    prompts = []
    for input_text in df[coluna]:
        
    # Crie a string de instrução
        test_instr = {
            "role": "user",
            "content": input_text
        }
        payload_temp = payload.copy()
        payload_temp['messages'] = payload_temp["messages"] + [test_instr]

        prompts.append(payload_temp)
    
    return prompts

In [10]:
# output_processado = []
# for prompt in tqdm(prompts, bar_format='{l_bar}{bar:10}{r_bar}{bar:-10b}'):
# # Send request
#     try:
#         response = requests.post(GPT4V_ENDPOINT, headers=headers, json=prompt)
#         response.raise_for_status()  # Will raise an HTTPError if the HTTP request returned an unsuccessful status code
#     except requests.RequestException as e:
#         raise SystemExit(f"Failed to make the request. Error: {e}")
    
#     output_processado.append(response.json()['choices'][0]['message']['content'])


In [7]:
def processar_gpt(lista_prompt):
    output_processado = []
    for prompt in tqdm(lista_prompt, bar_format='{l_bar}{bar:10}{r_bar}{bar:-10b}'):
    # Send request
        try:
            response = requests.post(GPT4V_ENDPOINT, headers=headers, json=prompt)
            response.raise_for_status()  # Will raise an HTTPError if the HTTP request returned an unsuccessful status code
        except requests.RequestException as e:
            raise SystemExit(f"Failed to make the request. Error: {e}")
        
        output_processado.append(response.json()['choices'][0]['message']['content'])

    return output_processado


# Lista erradas

In [ ]:
# Payload for the request
payload_incorreto = {
  "messages": [
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": "You're the new maintenance inspector at an aviation company.\n\nAs you're new to the job, you're still making mistakes and don't know exactly all the correct procedures.\n\nI'm going to go through inspection items from an aircraft maintenance plan and I want you to generate an incorrect inspection description based on the text I've given you.\nYou can't answer like a relaxed person who hasn't bothered to look properly.\nUse the item I've given you as a basis, but consider that you didn't have access to the maintenance plan, but didn't know you were doing it wrong.\n\nAll texts must be answered impersonally."
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "Ventilation System - Inspect clamps, hoses, and valves for condition and security."
        }
      ]
    },
    {
      "role": "assistant",
      "content": [
        {
          "type": "text",
          "text": "Inspected the clamps, cables, power panels and the general condition of the ventilation system."
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "Interior Placards, Exterior Placards, Decals, Markings and Identification Plates - Examine for correct installation and legibility. Refer to Chapter 11 Placards and Markings - Inspection/Check.\n"
        }
      ]
    },
    {
      "role": "assistant",
      "content": [
        {
          "type": "text",
          "text": "Visual inspection of external and identification Placards. Correct installation criteria checked using chapter 13.\n"
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "Cold and Hot Air Hoses - Check condition, routing, and security.\n"
        }
      ]
    },
    {
      "role": "assistant",
      "content": [
        {
          "type": "text",
          "text": "Inspect the general area around the air hoses. Check that there are no visible leaks. Nothing unusual observed."
        }
      ]
    }
  ],
  "temperature": 0.9,
  "top_p": 0.95,
  "max_tokens": 800
}

In [10]:
df_task['task_errada'] = output_processado

In [11]:
df_task

,TASK,task_errada
0,"Interior Placards, Exterior Placards, Decals, ...",Reviewed the placards for any signs of damage....
1,"Ventilation System - Inspect clamps, hoses, an...",Reviewed the overall functionality of the vent...
2,Primary Flight Display (PFD) Fan and Multi-Fun...,Conducted a visual inspection of the display f...
3,"Cold and Hot Air Hoses - Check condition, rout...",Examine the air hose system for any signs of w...
4,"Heater Components, Inlets, and Outlets - Inspe...",Checked the heater components and noted the in...
...,...,...
147,Exhaust System - Inspect for cracks and securi...,Conducted a visual inspection of the exhaust s...
148,Engine Oil - Drain oil sump and oil cooler. Ch...,Checked the engine oil level and ensured the o...
149,"Oil Cooler - Check for obstructions, leaks, an...",Inspected the oil cooler for general cleanline...
150,Starter and Electrical Connections - Check sec...,Reviewed the starter and noted its position. C...


In [12]:
df_task.to_excel('tasks_erradas.xlsx')

# lista correta

In [11]:
payload_correto = {
  "messages": [
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": """You're the maintenance inspector at an aviation company.
          \n\n\nI'm going to pass on inspection items from an aircraft maintenance plan and I want you to generate an inspection description based on the text I've given you.
          \n\nYou are a very experienced inspector and you carry out the items in the maintenance plan very carefully.
          \n\nAll texts must be answered impersonally."""
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "Interior Placards, Exterior Placards, Decals, Markings and Identification Plates - Examine for correct installation and legibility. Refer to Chapter 11 Placards and Markings - Inspection/Check.\n"
        }
      ]
    },
    {
      "role": "assistant",
      "content": [
        {
          "type": "text",
          "text": "After inspecting the Interior Placards, Exterior Placards, Decals, Markings and Identification Plates section, I verified that all components are in proper condition and secured.\n"
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "Restraint System, front and rear - Check belts for thinning, fraying, cutting, broken stitches, or ultra-violet deterioration. Check system hardware for security of installation.\n"
        }
      ]
    },
    {
      "role": "assistant",
      "content": [
        {
          "type": "text",
          "text": "Examined the Restraint System, front and rear for proper functioning and security; all looks satisfactory."
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "Cockpit Mounted Halon Type Fire Extinguishers - Empty, inspect for damage, and recharge.\n\n"
        }
      ]
    },
    {
      "role": "assistant",
      "content": [
        {
          "type": "text",
          "text": "Inspected the Cockpit Mounted Halon Type Fire Extinguishers thoroughly, ensuring all parts are operational and free from damage."
        }
      ]
    }
  ],
  "temperature": 0.9,
  "top_p": 0.95,
  "max_tokens": 800
}

In [12]:
prompts_correto = gerar_prompt(df_task,'TASK',payload_correto)

In [ ]:
lista_inspetor = processar_gpt(prompts_correto)

In [26]:
lista_inspetor[:3]

['Conducted a thorough examination of the Interior Placards, Exterior Placards, Decals, Markings, and Identification Plates. Verified correct installation and ensured that all markings are legible, in accordance with Chapter 11 Placards and Markings - Inspection/Check guidelines.',
 'Conducted a comprehensive inspection of the Ventilation System, assessing all clamps, hoses, and valves for integrity and secure installation. All components were found to be in satisfactory condition.',
 'Conducted an operational check on the Primary Flight Display (PFD) Fan, Multi-Function Display (MFD) Fan, Deck Skin Fan, and Remote Avionics Cooling Fan. All fans were found to be functioning within the specified parameters as outlined in Chapter 21, Avionics Cooling - Maintenance Practices.']

In [25]:
df_task['task_correta'] = lista_inspetor

# apgar

In [10]:
df_real = pd.read_excel('Listas_geradas.xlsx',sheet_name='Sheet1')

In [12]:
df_real

,task_normativo,inspec_correta,inspec_correta_gramatica,inspec_parcial,inspec_parcial_gramatica,inspec_errada
0,"Interior Placards, Exterior Placards, Decals, ...",Conducted a thorough examination of the Interi...,Conducted a thru examination of the Intierior ...,Reviewed the interior and exterior placards. C...,Revued the interrior and exterer placards. Con...,Reviewed the placards for any signs of damage....
1,"Ventilation System - Inspect clamps, hoses, an...",Conducted a comprehensive inspection of the Ve...,Conducted a comprhensive inspecshun of the Ven...,Inspected the ventilation system. Checked a fe...,Inspected the vent system. Checked a few clamp...,Reviewed the overall functionality of the vent...
2,Primary Flight Display (PFD) Fan and Multi-Fun...,Conducted an operational check on the Primary ...,Conduccted an operashunal chek on the Primery ...,Conducted an operational check on the PFD fan....,Conducted an oprational chek on the PFD fan. I...,Conducted a visual inspection of the display f...
3,"Cold and Hot Air Hoses - Check condition, rout...",Inspected Cold and Hot Air Hoses for overall c...,Inspted Cold an Hot Air Hozes for overal condi...,Inspected the cold and hot air hoses. Confirme...,Inspekted the cold and hot air hosess. Confirm...,Examine the air hose system for any signs of w...
4,"Heater Components, Inlets, and Outlets - Inspe...",Conducted a comprehensive inspection of Heater...,Condukted a comprehensiv inspction of Htr Comp...,Inspected heater components and inlets. Checke...,Inspected heatar compnents and inletz. Checked...,Checked the heater components and noted the in...
...,...,...,...,...,...,...
147,Exhaust System - Inspect for cracks and securi...,Conducted a detailed inspection of the Exhaust...,Conducted a detaled inspecshun of the Exhast S...,Inspected the exhaust system for cracks. Check...,Inspected the exahust system for cracks. Chcke...,Conducted a visual inspection of the exhaust s...
148,Engine Oil - Drain oil sump and oil cooler. Ch...,Performed the oil change procedure by draining...,Perfrmed the oil chng procedur by draining the...,The oil change procedure was noted. The oil fi...,The oil chnge procidur was notd. The oil filtt...,Checked the engine oil level and ensured the o...
149,"Oil Cooler - Check for obstructions, leaks, an...",Conducted a comprehensive inspection of the Oi...,Conducdted a comprensive inspecshun of the OIL...,Inspected the oil cooler. Confirmed no obstruc...,Inspected the oil coller. Confirmed no obstruc...,Inspected the oil cooler for general cleanline...
150,Starter and Electrical Connections - Check sec...,Inspected the Starter and Electrical Connectio...,"Inspected the Startor and Elec. Connections, c...",Inspected the starter and electrical connectio...,Insptd the strtr and elctrcl conxns. Confirmd ...,Reviewed the starter and noted its position. C...


# lista parcial

In [ ]:
I'm going to give you information about an item in an aircraft's maintenance plan and the inspector's report for the item in question.
          Taking both pieces of information into account, you need to write a new inspection report removing most part of the checks the inspector carried out.
          All texts must be answered impersonally and short.

In [32]:
payload_parcial = {
  "messages": [
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": """I'm going to give you information about an item in an aircraft's maintenance plan and the inspector's report for the item in question.
          Taking both pieces of information into account, you need to write a new inspection report removing most part of the checks the inspector carried out.
          All texts must be answered impersonally and short."""
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "Conducted a thorough examination of the Interior Placards, Exterior Placards, Decals, Markings, and Identification Plates. Verified correct installation and ensured that all markings are legible, in accordance with Chapter 11 Placards and Markings - Inspection/Check guidelines."
        }
      ]
    },
    {
      "role": "assistant",
      "content": [
        {
          "type": "text",
          "text": "Carried out a thorough examination of internal signs and nameplates. Checked for correct installation in accordance inspection/verification guidelines."
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "Conducted a thorough inspection of the Power Junction Box, assessing its operational status and overall condition. Verified the availability and condition of the spare fuse, confirming compliance with standards."
        }
      ]
    },
    {
      "role": "assistant",
      "content": [
        {
          "type": "text",
          "text": "Carried out an inspection of the power junction. Checked the condition confirming compliance with standards."
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "Conducted a detailed inspection of the Crankcase, Oil Sump, and Accessory Section. No cracks or oil leakage were found. Bolts and nuts were assessed for looseness and re-torqued as required. Crankcase breather lines were checked for obstructions and overall condition, confirming they are secure and in acceptable condition.\n"
        }
      ]
    },
    {
      "role": "assistant",
      "content": [
        {
          "type": "text",
          "text": "The crankcase and accessory section were inspected. Bolts and nuts were checked for looseness and retightened. The crankcase vent lines were checked."
        }
      ]
    }
  ],
  "temperature": 0.8,
  "top_p": 0.5,
  "max_tokens": 800
}


In [39]:
payload_parcial = {
  "messages": [
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": """You're an assistant who has to help me with my degree work.

Your task is to simulate that you're a maintenance inspector who doesn't like his job, so you carry out all the inspections you're asked to with unwillingness and laziness.

For my work scenario, I'm going to give you an inspection plan for an airplane component. Based on the items and components in the inspection plan, you should only carry out one or two activities at most and return your inspection report.

Finally, your answers should be impersonal and straightforward."""
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "Interior Placards, Exterior Placards, Decals, Markings and Identification Plates - Examine for correct installation and legibility. Refer to Chapter 11 Placards and Markings - Inspection/Check."
        }
      ]
    },
    {
      "role": "assistant",
      "content": [
        {
          "type": "text",
          "text": "Carried out a examination of internal signs and nameplates. Checked for correct installation in accordance inspection/verification guidelines."
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "Power Junction Box - Check operation and condition. Check availability and condition of spare fuse (if applicable)."
        }
      ]
    },
    {
      "role": "assistant",
      "content": [
        {
          "type": "text",
          "text": "Carried out an inspection of the power junction. Checked the condition too."
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "Crankcase, Oil Sump, and Accessory Section - Inspect for cracks and evidence of oil leakage. Check bolts and nuts for looseness and retorque as necessary. Check crankcase breather lines for obstructions, security, and general condition."
        }
      ]
    },
    {
      "role": "assistant",
      "content": [
        {
          "type": "text",
          "text": "The crankcase and accessory section were inspected. Bolts and nuts were checked for looseness and retightened. The crankcase vent lines were checked."
        }
      ]
    }
  ],
  "temperature": 0.8,
  "top_p": 0.5,
  "max_tokens": 800
}


In [38]:
payload_parcial = {
  "messages": [
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": """You're an assistant who has to help me with my degree work.

Your task is to simulate that you're a maintenance inspector who doesn't like his job, so you carry out all the inspections you're asked to with unwillingness and laziness.

For my work scenario, I'm going to give you an inspection plan for an airplane component. Based on the items and components in the inspection plan, you should only carry out one or two activities at most and return your inspection report.

Finally, your answers should be impersonal and straightforward."""
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "Interior Placards, Exterior Placards, Decals, Markings and Identification Plates - Examine for correct installation and legibility. Refer to Chapter 11 Placards and Markings - Inspection/Check."
        }
      ]
    },
    {
      "role": "assistant",
      "content": [
        {
          "type": "text",
          "text": "Inspection on Decals and Identification plates."
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "Power Junction Box - Check operation and condition. Check availability and condition of spare fuse (if applicable)."
        }
      ]
    },
    {
      "role": "assistant",
      "content": [
        {
          "type": "text",
          "text": "Inspection of the power junction."
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "Crankcase, Oil Sump, and Accessory Section - Inspect for cracks and evidence of oil leakage. Check bolts and nuts for looseness and retorque as necessary. Check crankcase breather lines for obstructions, security, and general condition."
        }
      ]
    },
    {
      "role": "assistant",
      "content": [
        {
          "type": "text",
          "text": "The Oil Sump and accessory section were inspected."
        }
      ]
    }
  ],
  "temperature": 0.8,
  "top_p": 0.3,
  "max_tokens": 800
}


In [39]:
prompts_parcial = gerar_prompt(df_real,'inspec_correta',payload_parcial)

In [41]:
prompts_parcial[0]

{'messages': [{'role': 'system',
   'content': [{'type': 'text',
     'text': "You're an assistant who has to help me with my degree work.\n\nYour task is to simulate that you're a maintenance inspector who doesn't like his job, so you carry out all the inspections you're asked to with unwillingness and laziness.\n\nFor my work scenario, I'm going to give you an inspection plan for an airplane component. Based on the items and components in the inspection plan, you should only carry out one or two activities at most and return your inspection report.\n\nFinally, your answers should be impersonal and straightforward."}]},
  {'role': 'user',
   'content': [{'type': 'text',
     'text': 'Interior Placards, Exterior Placards, Decals, Markings and Identification Plates - Examine for correct installation and legibility. Refer to Chapter 11 Placards and Markings - Inspection/Check.'}]},
  {'role': 'assistant',
   'content': [{'type': 'text',
     'text': 'Inspection on Decals and Identificati

In [42]:
lista_parcial = processar_gpt(prompts_parcial)

100%|██████████| 152/152 [02:28<00:00,  1.02it/s]


In [44]:
lista_parcial[:15]

['Inspection of Interior and Exterior Placards completed. Markings verified for legibility.',
 'Inspected a few clamps and hoses of the Ventilation System.',
 'Operational check on the PFD Fan was conducted.',
 'Inspected the Cold and Hot Air Hoses. All found satisfactory.',
 'Inspected heater components and ducts.',
 'Inspected the Cabin Heat controls for movement.',
 'Only a basic check of the Autopilot Rigging was performed.',
 'Inspected the Autopilot Servo Capstan Assemblies. Verified slip-clutch torque settings.',
 'Inspected Autopilot Servo Actuators for corrosion. All components appeared clean.',
 'Inspection of Communication Antennas was completed. No issues found.',
 'Inspected the headsets for cleanliness and secure attachment.',
 'Inspected the Microphone Push-To-Talk Switches. Functions appear operational.',
 'Inspection of the alternator and mounting bracket was completed.',
 'Main Battery condition and security were checked.',
 'Main Battery Box and cables inspected. Con

In [45]:
df_task['task_parcial'] = lista_parcial

In [46]:
df_task.to_excel('parcial_correto3.xlsx')

In [47]:
df_task

,TASK,task_parcial,task_parcial_gramatica
0,"Interior Placards, Exterior Placards, Decals, ...",Inspection of Interior and Exterior Placards c...,Conducdted a basik chek of the interor and ext...
1,"Ventilation System - Inspect clamps, hoses, an...",Inspected a few clamps and hoses of the Ventil...,Inspected the ventlation sysmtem. Clamps and h...
2,Primary Flight Display (PFD) Fan and Multi-Fun...,Operational check on the PFD Fan was conducted.,Conducted an ops check on the PFD fan. The fan...
3,"Cold and Hot Air Hoses - Check condition, rout...",Inspected the Cold and Hot Air Hoses. All foun...,Insptd the hot air hozes. Found them to b in s...
4,"Heater Components, Inlets, and Outlets - Inspe...",Inspected heater components and ducts.,Inspted heator compnents and inlets. Evaluated...
...,...,...,...
147,Exhaust System - Inspect for cracks and securi...,Inspected the Exhaust System for cracks. Check...,Inspected the exaust systm for cracks n checke...
148,Engine Oil - Drain oil sump and oil cooler. Ch...,Oil change procedure noted. No contamination f...,The oil chng procedur was noted. The oil fltr ...
149,"Oil Cooler - Check for obstructions, leaks, an...",Inspection of the Oil Cooler was completed. No...,Insptd the Oil Clr. Confrmd no obstuctions or ...
150,Starter and Electrical Connections - Check sec...,Inspected the starter connections. All cables ...,Inspectd the stater and electrcal connections....


# erros_gramatica

In [48]:
payload_gramatica = {
  "messages": [
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": "I'm going to give you the report of the maintenance inspector of an aviation company.\n\nI want you to rewrite it with a few typos, grammatical errors and even aviation abbreviations.\n\nThe idea is to make the inspector's text more difficult, but not incomprehensible."
        }
      ]
    }
  ],
  "temperature": 0.9,
  "top_p": 0.95,
  "max_tokens": 800
}

In [49]:
prompts_correto_gramatica = gerar_prompt(df_task,'task_correta',payload_gramatica)

In [50]:
prompts_correto_gramatica[0]

{'messages': [{'role': 'system',
   'content': [{'type': 'text',
     'text': "I'm going to give you the report of the maintenance inspector of an aviation company.\n\nI want you to rewrite it with a few typos, grammatical errors and even aviation abbreviations.\n\nThe idea is to make the inspector's text more difficult, but not incomprehensible."}]},
  {'role': 'user',
   'content': 'Conducted a thorough examination of the Interior Placards, Exterior Placards, Decals, Markings, and Identification Plates. Verified correct installation and ensured that all markings are legible, in accordance with Chapter 11 Placards and Markings - Inspection/Check guidelines.'}],
 'temperature': 0.9,
 'top_p': 0.95,
 'max_tokens': 800}

In [51]:
lista_correto_gramatica = processar_gpt(prompts_correto_gramatica)

100%|██████████| 152/152 [03:42<00:00,  1.46s/it]


In [53]:
df_task['task_correta_gramatica'] = lista_correto_gramatica

In [49]:
prompts_parcial_gramatica = gerar_prompt(df_task,'task_parcial',payload_gramatica)


In [51]:
prompts_parcial_gramatica[0]

{'messages': [{'role': 'system',
   'content': [{'type': 'text',
     'text': "I'm going to give you the report of the maintenance inspector of an aviation company.\n\nI want you to rewrite it with a few typos, grammatical errors and even aviation abbreviations.\n\nThe idea is to make the inspector's text more difficult, but not incomprehensible."}]},
  {'role': 'user',
   'content': 'Inspection of Interior and Exterior Placards completed. Markings verified for legibility.'}],
 'temperature': 0.9,
 'top_p': 0.95,
 'max_tokens': 800}

In [52]:
lista_parcial_gramatica = processar_gpt(prompts_parcial_gramatica)

100%|██████████| 152/152 [02:23<00:00,  1.06it/s]


In [53]:
df_task['task_parcial_gramatica'] = lista_parcial_gramatica

In [66]:
df_task = df_task[['TASK','task_correta','task_correta_gramatica','task_parcial','task_parcial_gramatica','task_errada']]


In [67]:
df_task

,TASK,task_correta,task_correta_gramatica,task_parcial,task_parcial_gramatica,task_errada
0,"Interior Placards, Exterior Placards, Decals, ...",Conducted a thorough examination of the Interi...,Conducted a thru examination of the Intierior ...,Performed an examination of internal and exter...,Perfromed an exaimnation of internal and extte...,Reviewed the placards for any signs of damage....
1,"Ventilation System - Inspect clamps, hoses, an...",Conducted a comprehensive inspection of the Ve...,Conducted a comprhensive inspecshun of the Ven...,"Inspected the ventilation system, assessing ho...","Inspted the ventlation systm, assesing hoses n...",Reviewed the overall functionality of the vent...
2,Primary Flight Display (PFD) Fan and Multi-Fun...,Conducted an operational check on the Primary ...,Conduccted an operashunal chek on the Primery ...,Performed an operational check on the primary ...,Perforned an opreational chek on the primery f...,Conducted a visual inspection of the display f...
3,"Cold and Hot Air Hoses - Check condition, rout...",Inspected Cold and Hot Air Hoses for overall c...,Inspted Cold an Hot Air Hozes for overal condi...,Inspected cold and hot air hoses for overall c...,Inspected cold n hot air hosses for overal con...,Examine the air hose system for any signs of w...
4,"Heater Components, Inlets, and Outlets - Inspe...",Conducted a comprehensive inspection of Heater...,Condukted a comprehensiv inspction of Htr Comp...,Conducted an inspection of heater components a...,Conductd an inspction of heatr compnents and i...,Checked the heater components and noted the in...
...,...,...,...,...,...,...
147,Exhaust System - Inspect for cracks and securi...,Conducted a detailed inspection of the Exhaust...,Conducted a detaled inspecshun of the Exhast S...,"Inspected the exhaust system, focusing on the ...","Insptd the exaust system, focussing on the int...",Conducted a visual inspection of the exhaust s...
148,Engine Oil - Drain oil sump and oil cooler. Ch...,Performed the oil change procedure by draining...,Perfrmed the oil chng procedur by draining the...,"Executed the oil change procedure, including d...","Execuited the oil change prcedure, including d...",Checked the engine oil level and ensured the o...
149,"Oil Cooler - Check for obstructions, leaks, an...",Conducted a comprehensive inspection of the Oi...,Conducdted a comprensive inspecshun of the OIL...,"Inspected the oil cooler, confirming the absen...","Inspted the oil cooler, confiming the absense ...",Inspected the oil cooler for general cleanline...
150,Starter and Electrical Connections - Check sec...,Inspected the Starter and Electrical Connectio...,"Inspected the Startor and Elec. Connections, c...",Inspected the starter and electrical connectio...,"Insptd the strtr and electrcl cnxns, cnfrming ...",Reviewed the starter and noted its position. C...


In [54]:
df_task.to_excel('Listas_geradas222.xlsx')